<a href="https://colab.research.google.com/github/RSN601KRI/Water_potability_ML_Prblm/blob/main/MLPrblm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [8]:
!pip install pandas numpy scikit-learn

In [9]:
import pandas as pd

# Load CSV files
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print("Train Dataset:")
print(train_df.head())

print("\nTest Dataset:")
print(test_df.head())

Train Dataset:
   Index                                                 ph  \
0      0  <p>😅The pH value observed is:  0.2644088544860...   
1      1  <p>😇The solution's pH level is  0.012034498894...   
2      2  <br>😀The pH reading measures 0.491167423175516...   
3      3  <h1>😅The water sample's pH level is  0.4865531...   
4      4  <p>😍The observed pH value of  0.07548804288855...   

                                            Hardness  \
0  <br>😒With a hardness level of  2.6735383052025...   
1  <h1>😀The water hardness is measured at  2.6303...   
2  <h2>😅With a hardness level of  1.2213782428674...   
3  <h2>😀With a hardness level of  1.2591335755542...   
4  <p>😁With a hardness level of  1.65681755735377...   

                                              Solids  Chloramines   Sulfate  \
0  <br>😅The water sample exhibits a total solids ...     1.431972  1.303685   
1  <br>😒The total solids content in the sample is...     0.911372  1.181754   
2  <h1>😍The water sample exhibit

In [10]:
import re
import numpy as np

# Sample test cases
sample_values = [
    "<h1>😍The water hardness is measured at  1.3005608204573031  ...</h1>",
    "The pH value is <b>7.2</b>",
    "Invalid text with 💧",
    "Hardness level: -145.67 detected"
]

# Cleaning function
def extract_numeric_value(text):
    clean_text = re.sub(r"<[^>]*>|[^\d\.\-]", "", str(text))  # Remove HTML and non-numeric characters
    match = re.search(r"(-?\d+\.?\d*)", clean_text)  # Match numeric values
    return float(match.group(1)) if match else np.nan

# Apply cleaning to test cases
for val in sample_values:
    print(f"Original: {val} --> Cleaned: {extract_numeric_value(val)}")

Original: <h1>😍The water hardness is measured at  1.3005608204573031  ...</h1> --> Cleaned: 1.3005608204573031
Original: The pH value is <b>7.2</b> --> Cleaned: 7.2
Original: Invalid text with 💧 --> Cleaned: nan
Original: Hardness level: -145.67 detected --> Cleaned: -145.67


In [11]:
train_df["ph"] = train_df["ph"].apply(extract_numeric_value)
print(train_df["ph"].head())

# Check for any invalid (NaN) values
print("\nNumber of NaN values in 'ph':", train_df["ph"].isna().sum())

0    0.264409
1    0.012034
2    0.491167
3    0.486553
4    0.075488
Name: ph, dtype: float64

Number of NaN values in 'ph': 0


In [14]:
import re
import numpy as np

# Function to extract numeric values from strings
def extract_numeric(text):
    match = re.search(r"(\d+\.?\d*)", str(text))  # Extract numeric values with decimals
    if match:
        return float(match.group(1))  # Return as float
    else:
        return np.nan  # Return NaN if no match

# Apply extraction to all numeric columns in train and test data
for col in train_df.columns:
    train_df[col] = train_df[col].apply(extract_numeric)

for col in test_df.columns:
    test_df[col] = test_df[col].apply(extract_numeric)

In [16]:
# Fill NaN values with median for train data
numeric_features = train_df.select_dtypes(include=np.number).columns.tolist()
train_df[numeric_features] = train_df[numeric_features].fillna(train_df[numeric_features].median())

# Fill NaN values for test data (excluding Potability)
numeric_features_test = [col for col in numeric_features if col in test_df.columns]
test_df[numeric_features_test] = test_df[numeric_features_test].fillna(train_df[numeric_features_test].median())

In [17]:
# Separate features (X) and target (y)
X = train_df.drop(columns=["Potability", "Index"], errors="ignore")
y = train_df["Potability"]

# Clean test data
X_test = test_df.drop(columns=["Index"], errors="ignore")

In [20]:
# Step 5: Ensure the target variable is binary (0 or 1)
# Round or threshold continuous values to 0 and 1
train_df["Potability"] = train_df["Potability"].apply(lambda x: 1 if x >= 0.5 else 0)

# Step 6: Prepare features and target
X = train_df.drop(columns=["Potability", "Index"], errors="ignore")
y = train_df["Potability"]

# Prepare test data
X_test = test_df.drop(columns=["Index"], errors="ignore")

# Step 7: Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Train RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 9: Validate model
y_val_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Step 10: Predict on test data
test_predictions = model.predict(X_test)

# Step 11: Prepare submission file
submission = pd.DataFrame({
    "Index": test_df["Index"],
    "Potability": test_predictions
})

# Save submission to CSV
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

Validation Accuracy: 0.9865
Submission file saved as submission.csv
